For this journal we will attempt our first prediction models.
As always, let's import our libraries

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm
import statsmodels.api as sm
from statsmodels.stats.proportion import proportions_ztest
import mpl_toolkits.mplot3d as m3d
from scipy import special
import os

Now let's import our dataset, which is the dataset we created at the end of our previous journal

In [27]:
df = pd.read_csv('/Users/lokikeeler/Downloads/train-balanced-sarcasm_2.csv')
df.head()

,label,comment,score,ups,downs,date,created_utc,parent_comment,year,SUB_2007scape,...,SUB_television,SUB_tf2,SUB_todayilearned,SUB_trees,SUB_ukpolitics,SUB_unitedkingdom,SUB_videos,SUB_worldnews,SUB_wow,SUB_xboxone
0,0,NC and NH.,2,-1,-1,2016-10-01,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ...",2016,False,...,False,False,False,False,False,False,False,False,False,False
1,0,You do know west teams play against west teams...,-4,-1,-1,2016-11-01,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...,2016,False,...,False,False,False,False,False,False,False,False,False,False
2,0,"They were underdogs earlier today, but since G...",3,3,0,2016-09-01,2016-09-22 21:45:37,They're favored to win.,2016,False,...,False,False,False,False,False,False,False,False,False,False
3,0,"This meme isn't funny none of the ""new york ni...",-8,-1,-1,2016-10-01,2016-10-18 21:03:47,deadass don't kill my buzz,2016,False,...,False,False,False,False,False,False,False,False,False,False
4,0,"I don't pay attention to her, but as long as s...",0,0,0,2016-09-01,2016-09-02 10:35:08,do you find ariana grande sexy ?,2016,False,...,False,False,False,False,False,False,False,False,False,False


For our first logistic regression model we need to edit our columns so we don't include object data types and also prevent colinearity. Comment and parent comment are object data types. Also we have year, so we don't need date and created_utc.

In [28]:
df_2 = df.drop(['comment', 'date', 'created_utc', 'parent_comment'], axis=1)
df_2.head()

,label,score,ups,downs,year,SUB_2007scape,SUB_AdviceAnimals,SUB_Android,SUB_AskMen,SUB_AskReddit,...,SUB_television,SUB_tf2,SUB_todayilearned,SUB_trees,SUB_ukpolitics,SUB_unitedkingdom,SUB_videos,SUB_worldnews,SUB_wow,SUB_xboxone
0,0,2,-1,-1,2016,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,0,-4,-1,-1,2016,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,0,3,3,0,2016,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,0,-8,-1,-1,2016,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,0,0,0,0,2016,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


Great. Now let's pull up some initial correlation values

In [29]:
df_2.corr().style.background_gradient()

Looking at the first column of correlations with Label, we can see all correlations are very small. The greatest in a negative direction is the subreddit Ask Reddit with -0.09. Other higher correlations are about 0.05. Overall, these are very low

Let's turn our dummy columns into floats so our model can work

In [30]:
df_2 = df_2.astype(np.float64)

Now let's define X as all the columns right of Label, and our y as label. Then lets add the constant

In [31]:
X = df_2[df_2.columns[1:]]
y = df_2['label']

X_withconstant = sm.add_constant(X)

X_withconstant.head()

,const,score,ups,downs,year,SUB_2007scape,SUB_AdviceAnimals,SUB_Android,SUB_AskMen,SUB_AskReddit,...,SUB_television,SUB_tf2,SUB_todayilearned,SUB_trees,SUB_ukpolitics,SUB_unitedkingdom,SUB_videos,SUB_worldnews,SUB_wow,SUB_xboxone
0,1.0,2.0,-1.0,-1.0,2016.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,-4.0,-1.0,-1.0,2016.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,3.0,3.0,0.0,2016.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,-8.0,-1.0,-1.0,2016.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,2016.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Next let's create our logistic model and fit the model.

In [32]:
mylogreg= sm.Logit(y,X_withconstant)
mylogreg_results = mylogreg.fit()
mylogreg_results.summary()

Optimization terminated successfully.
         Current function value: 0.678544
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  label   No. Observations:               549904
Model:                          Logit   Df Residuals:                   549799
Method:                           MLE   Df Model:                          104
Date:                Tue, 02 Jan 2024   Pseudo R-squ.:                 0.01926
Time:                        22:09:28   Log-Likelihood:            -3.7313e+05
converged:                       True   LL-Null:                   -3.8046e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     113.1000        nan        nan        nan         nan         nan
score                      -0.0012      0.002     -0.651      0.515      -0.005       0.002
ups                        -0.0124      0.002     -6.292      0.000      -0.016      -0.009
downs                       0.2853      0.011     25.194      0.000       0.263       0.307
year                       -0.0566      0.002    -23.489      0.000      -0.061      -0.052
SUB_2007scape               1.1100        nan        nan        nan         nan         nan
SUB_AdviceAnimals           1.2114        nan        nan        nan         nan         nan
SUB_Android                 1.0090        nan        nan        nan         nan         nan
SUB_AskMen                  0.7872        nan        nan        nan         nan         nan
SUB_AskReddit               0.6167        nan        nan        nan         nan         nan
SUB_BigBrother              1.0647        nan        nan        nan         nan         nan
SUB_Bitcoin                 1.1378        nan        nan        nan         nan         nan
SUB_BlackPeopleTwitter      0.6272        nan        nan        nan         nan         nan
SUB_CFB                     1.0750        nan        nan        nan         nan         nan
SUB_CODZombies              1.1417        nan        nan        nan         nan         nan
SUB_Christianity            1.2225        nan        nan        nan         nan         nan
SUB_CoDCompetitive          1.2641        nan        nan        nan         nan         nan
SUB_Conservative            1.5915        nan        nan        nan         nan         nan
SUB_CringeAnarchy           0.9921        nan        nan        nan         nan         nan
SUB_DestinyTheGame          1.1352        nan        nan        nan         nan         nan
SUB_DotA2                   0.9956        nan        nan        nan         nan         nan
SUB_Eve                     0.6814        nan        nan        nan         nan         nan
SUB_Futurology              1.3231        nan        nan        nan         nan         nan
SUB_GlobalOffensive         1.2257        nan        nan        nan         nan         nan
SUB_KotakuInAction          1.2998        nan        nan        nan         nan         nan
SUB_Libertarian             1.5093        nan        nan        nan         nan         nan
SUB_MMA                     1.1375        nan        nan        nan         nan         nan
SUB_MensRights              1.6609        nan        nan        nan         nan         nan
SUB_Monstercat              1.1658        nan        nan        nan         nan         nan
SUB_NoMansSkyTheGame        1.2206        nan        nan        nan         nan         nan
SUB_Overwatch               0.9479        nan        nan        nan         nan         nan
SUB_PS4                     0.9700        nan        nan        nan         nan         nan
SUB_Planetside       

Off first glance we see many null values for the subreddits. Furthermore, we can see a low R squared value. On the other hand, we see a low P value, thus statiscal significance, with ups, downs, and year

Let's see how our predictions are with this model. We will base anything over 0.5 to be 1 (sarcastic) and all else 0 (not sarcastic).

In [33]:
model_predictions_prob = mylogreg_results.predict(X_withconstant)
model_predictions_binary = np.where(model_predictions_prob>0.5,1,0)

In [34]:
(model_predictions_binary == df_2['label']).sum()

313634

In [35]:
len(df_2['label'])

549904

In [36]:
print("The accuracy is:", (313634/549904)*100 , "%")

The accuracy is: 57.03431871744886 %


Our initial model is pretty bad, with 57%. Let's see if we can improve upon this. For this next one, why don't we try working with columns that had at least 0.015 correlation (positive or negative)

In [38]:
df_3 = df_2[['label', 'ups', 'downs', 'year', 'score', 'SUB_AskReddit', 'SUB_atheism', 'SUB_aww', 
            'SUB_creepyPMs', 'SUB_fantasyfootball', 'SUB_news', 'SUB_worldnews',
            'SUB_MensRights']]

In [39]:
df_3.head()

,label,ups,downs,year,score,SUB_AskReddit,SUB_atheism,SUB_aww,SUB_creepyPMs,SUB_fantasyfootball,SUB_news,SUB_worldnews,SUB_MensRights
0,0.0,-1.0,-1.0,2016.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,-1.0,-1.0,2016.0,-4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,3.0,0.0,2016.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,-1.0,-1.0,2016.0,-8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,2016.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now let's repeat the process. Defining X and y. Adding the constant. Fitting the model.

In [40]:
X = df_3[df_3.columns[1:]]
y = df_3['label']

X_withconstant_2 = sm.add_constant(X)

In [41]:
mylogreg_2= sm.Logit(y,X_withconstant_2)
mylogreg_2_results = mylogreg_2.fit()
mylogreg_2_results.summary()

Optimization terminated successfully.
         Current function value: 0.682660
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  label   No. Observations:               549904
Model:                          Logit   Df Residuals:                   549891
Method:                           MLE   Df Model:                           12
Date:                Tue, 02 Jan 2024   Pseudo R-squ.:                 0.01331
Time:                        22:12:23   Log-Likelihood:            -3.7540e+05
converged:                       True   LL-Null:                   -3.8046e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                 101.3647      4.618     21.950      0.000      92.314     110.416
ups                    -0.0122      0.002     -6.220      0.000      -0.016      -0.008
downs                   0.2791      0.011     24.909      0.000       0.257       0.301
year                   -0.0502      0.002    -21.909      0.000      -0.055      -0.046
score                   0.0006      0.002      0.305      0.761      -0.003       0.004
SUB_AskReddit          -0.5150      0.009    -58.956      0.000      -0.532      -0.498
SUB_atheism             0.3789      0.025     15.099      0.000       0.330       0.428
SUB_aww                -0.8627      0.045    -19.256      0.000      -0.951      -0.775
SUB_creepyPMs           1.1423      0.036     31.814      0.000       1.072       1.213
SUB_fantasyfootball    -0.5641      0.044    -12.722      0.000      -0.651      -0.477
SUB_news                0.2936      0.017     17.686      0.000       0.261       0.326
SUB_worldnews           0.4483      0.014     32.862      0.000       0.422       0.475
SUB_MensRights          0.5326      0.039     13.833      0.000       0.457       0.608
=======================================================================================
"""

These results are interesting. The R Squared is still small, but now all columns (except score) have a lot P value, showing statistical significance. Also there are no null values. Now, let's check the accuracy


In [42]:
model_predictions_prob_2 = mylogreg_2_results.predict(X_withconstant_2)
model_predictions_binary_2 = np.where(model_predictions_prob_2>0.5,1,0)

In [43]:
(model_predictions_binary_2 == df_3['label']).sum()

305322

In [44]:
len(df_3['label'])

549904

In [45]:
print("The accuracy is:", (305322/549904)*100 , "%")

The accuracy is: 55.52278215834037 %


Hmm. Unfortunately, our model is even worse. Back to the drawing board.